In [1]:
!pip install email_validator

In [2]:
#import libraries
from datetime import date, datetime
import requests
from email_validator import validate_email
import pandas as pd
import getpass
import pyodbc

In [3]:
#create connection 
# Replace the input connection string with your actual connection string
connection_string = 'Server=tcp:bbyc-sqlsv-prod-database-stg-005.database.windows.net,1433;Initial Catalog=bbyc-sqldb-imidiapf-stg-001;Persist Security Info=False;MultipleActiveResultSets=False;Encrypt=True;TrustServerCertificate=False;'

# Replace 'your_username' and 'your_password' with your actual AD username and password
ad_username = 'arbanerj@bbyfs.ca' #change it to your username 
ad_password = getpass.getpass() #this will ask for your password

modified_connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={connection_string.split(";")[0].split("=")[1]};DATABASE={connection_string.split(";")[1].split("=")[1]};Authentication=ActiveDirectoryPassword;UID={ad_username};PWD={ad_password};'

# Establish the connection
connection = pyodbc.connect(modified_connection_string)
print('connected ...')

 ········


connected ...


In [4]:
class EmailValidator:
  def __init__(
        self,
        connection,
        params):
        self.params = params
        self.connection = connection
        self.database = params['DATABASE_NAME']
        self.schema = params['SCHEMA_NAME']
        self.table = params['TABLE_NAME']
        self.column = params['COLUMN_NAME']
        self.where = params['ISSCURRENT']
        self.drive_path = params['DRIVE_PATH']
        self.who = params['WHO']
        self.when = params['WHEN']
        self.location  = params['WHERE']
        self.how =  params['HOW']

  def getTotalRecordCount(self):
        record_count_sql = (
            f"SELECT COUNT(*) FROM \
                            {self.schema}.{self.table}"
        )
        record_count = self.connection.cursor().execute(record_count_sql).fetchall()
        return record_count[0][0]

  def getTotalNotNullOrBlankEmail(self):
        not_null_count_sql = f"SELECT COUNT(*) FROM \
                             {self.schema}.{self.table} \
                             WHERE {self.column} IS NOT NULL AND LEN(LTRIM(RTRIM({self.column}))) > 0 "
        not_null_count = (
                self.connection.cursor().execute(not_null_count_sql).fetchall()
            )
        return not_null_count[0][0]

  def getCurrentDateTime(self):
        current_time_sql = "SELECT GETDATE();"
        current_time = self.connection.cursor().execute(current_time_sql).fetchall()
        return current_time[0][0]

  def validInvalidEmailCount(self):
        email_fetch_sql = f"SELECT {self.column} FROM \
                                  {self.schema}.{self.table} \
                                  WHERE {self.column} IS NOT NULL AND LEN(LTRIM(RTRIM({self.column}))) > 0 "
        emails = self.connection.cursor().execute(email_fetch_sql).fetchall()
        vailid_email_list= []
        invalid_email_list= []
        valid_email_count= 0
        invalid_email_count= 0
        count = 0
        for email in emails:
          try:
            if (len(email[0].split(".")[-1])) >1 and (len(email[0].split(".")[-1])) <=11 and (len(email[0].split("@")[0])) >=2 : #here top level domain considerd 2-6 character long. like .com,.gov,.in,.daomain
             validate_email(email[0].strip(), allow_smtputf8=False, check_deliverability=False)
             vailid_email_list.append(email[0])
             valid_email_count = valid_email_count + 1
            else:
             invalid_email_list.append(email[0])
             invalid_email_count = invalid_email_count + 1
          except:
            invalid_email_list.append(email[0])
            invalid_email_count = invalid_email_count + 1

        return vailid_email_list,invalid_email_list,valid_email_count,invalid_email_count

  def validInnvalidToCsv(self,vailid_email_list,invalid_email_list ):
    df = pd.DataFrame.from_dict({'Sample_Of_Valid_Email_IDs': vailid_email_list , 'Sample_Of_InValid_Email_IDs': invalid_email_list }, orient='index')
    df= df.transpose()
    df.to_csv(f'{self.drive_path}\\{self.database}_{self.schema}_{self.table}_{self.column}_VALID_INVALID_EMAIL.csv',index = False)
    return df

  def emailDomainGroupBy(self,df):
    df2 = pd.DataFrame(df['Sample_Of_Valid_Email_IDs'].str.strip().split('@',1).tolist(),columns = ['Name','Domain'])
    df3 = df2.groupby('Domain').count()
    df3.columns = ['Count']
    df3 = df3.sort_values(by='Count', ascending=False)
    df3.head(20).to_csv(f'{self.drive_path}\\{self.database}_{self.schema}_{self.table}_{self.column}_VALID_EMAIL_DOMAIN_COUNT.csv')

  def generalInfo(self,current_time,total_number_of_record,total_number_of_empty_email,percent_of_empty_email,valid_email_count,percent_of_valid_email_count,invalid_email_count,percent_of_invalid_email_count):
        print('general info....')
        datasummary = []
        datasummary.append(
            {
                "DATA SET NAME": self.table,
                "COLUMN NAME": self.column,
                "AS OF DATE": current_time,
                "RECORD COUNT": total_number_of_record,
                "NUMBER OF BLANK EMAIL ": str(total_number_of_empty_email)+'('+ str(round(percent_of_empty_email,4))+')%',
                "NUMBER OF INVALID EMAIL (APPROX)": str(invalid_email_count)+'('+ str(round(percent_of_invalid_email_count,4))+')%',
                "NUMBER OF VALID EMAIL (APPROX)": str(valid_email_count)+'('+ str(round(percent_of_valid_email_count,4))+')%',
                "WHO": self.who,
                "WHEN": self.when,
                "WHERE": self.location,
                "HOW": self.how
            }
        )
        data_frame = pd.DataFrame(datasummary)
        data_frame_transpose = data_frame .T.reset_index().rename(columns={'index': 'ATTRIBUTE', 0: 'VALUE'})
        gdrive_path = (
            f"{self.drive_path}\\{self.database}_{self.schema}_{self.table}_{self.column}_EMAIL_SUMMARY.csv"
        )
        data_frame_transpose.to_csv(gdrive_path, index=False)


  def getSummary(self):
    total_number_of_record = self.getTotalRecordCount()
    total_number_of_empty_email = total_number_of_record - self.getTotalNotNullOrBlankEmail()
    print(total_number_of_empty_email)
    percent_of_empty_email = (total_number_of_empty_email / total_number_of_record)*100
    vailid_email_list,invalid_email_list,valid_email_count,invalid_email_count = self.validInvalidEmailCount()
    percent_of_valid_email_count = (valid_email_count / total_number_of_record)*100
    percent_of_invalid_email_count = (invalid_email_count / total_number_of_record)*100
    df = self.validInnvalidToCsv(vailid_email_list,invalid_email_list)
    #self.emailDomainGroupBy(df)
    current_time = self.getCurrentDateTime()
    self.generalInfo(current_time,total_number_of_record,total_number_of_empty_email,percent_of_empty_email,valid_email_count,percent_of_valid_email_count,invalid_email_count,percent_of_invalid_email_count)


In [5]:
#get datetime of this machine
from datetime import datetime, timezone

def getDateTimeAsString():
  """
  This function gets the current date and time as a string with the timezone information.

  Returns:
      str: The current date and time as a string in the format YYYY-MM-DD HH:MM:SS with timezone offset (e.g., 2023-11-19 16:02:21+05:30).
  """
  now = datetime.now(timezone.utc)
  # Convert to local timezone for display
  local_time = now.astimezone()
  date_time_string = local_time.strftime("%Y-%m-%d %H:%M:%S%z")
  return date_time_string

# Print the current date and time with timezone
print(getDateTimeAsString())

2024-03-20 23:53:29-0700


In [6]:
params = {
            'SYSTEM_NAME': 'Accounts', # CHANGE THIS ONE AS PER REQUIRMENT
            'DATABASE_NAME': 'bbyc-sqldb-imidiapf-stg-001', # CHANGE THIS ONE AS PER REQUIRMENT
            'SCHEMA_NAME': 'temp',  # CHANGE THIS ONE AS PER REQUIRMENT
            'TABLE_NAME': 'temp_UserObject',  # CHANGE THIS ONE AS PER REQUIRMENT
            'COLUMN_NAME': 'u_email_address',
            'ISSCURRENT': "No filter has been added",
            'DRIVE_PATH': 'C:\\Users\\arbanerj_a\\',  # CHANGE THIS ONE AS PER REQUIRMENT
            'WHO': 'Arpan Banerjee', # CHANGE THIS ONE AS PER REQUIRMENT
            'WHEN': getDateTimeAsString(),
            'WHERE': 'Kolkata-INDIA', # CHANGE THIS ONE AS PER REQUIRMENT
            'HOW': '1. Python Snowflake Connecto and Python email-validator libray has been used \n 2. Here is the link to the documentation https://pypi.org/project/email-validator/ \n 3. allow_smtputf8 set to False which will exclude internationalized forms, more in the above link. \n 4. Same as general profile we have is ISCURRENT to TRUE'
}

In [7]:
email = EmailValidator(connection,params)
email.getSummary()

0
general info....


In [ ]:
connection.close()

In [ ]:
try:
    validate_email('LRDALESSIO@ROGERS.COM    '.strip(), allow_smtputf8=False, check_deliverability=False)
    print('is this valid?')
except:
    print('not an valid one')
    

In [ ]:
email[0] = 'X@C.COM'
if (len(email[0].split(".")[-1])) >1 and (len(email[0].split(".")[-1])) <=11 and (len(email[0].split("@")[0])) <=2 and not(email[0].split("@")[0]).isdigit():    try
      validate_email(email[0], allow_smtputf8=False, check_deliverability=False)
      print('is this valid?')
else: 
    print('not even enterd ')

In [ ]:
ema= 'X@C.COM'
(len(ema.split("@")[0])) >=2 and not(ema.split("@")[0]).isdigit()

In [ ]:
ema = '123@C.COM'
if (len(ema.split(".")[-1])) >1 and (len(ema.split(".")[-1])) <=11 and (len(ema.split("@")[0])) >=2 and (not(ema.split("@")[0]).isdigit()):    
      validate_email(ema, allow_smtputf8=False, check_deliverability=False)
      print('is this valid?')
else: 
    print('not even enterd ')

In [ ]:
!pip install phonenumbers

In [20]:
import re
import pandas as pd

def generate_pattern(phone_number):
    pattern = ""
    
    for char in phone_number:
        if char.isdigit():
            pattern += 'D'  # replace numeric value with D
        elif char.isalpha():
            pattern += 'A'  # replace alphabet with A
        elif char.isspace():
            pattern += ' '
        else:
            pattern += 'S'  # replace all other with S
    
    return pattern




In [25]:
pattern_counts = {}
phone_sql = f"SELECT  u_postal_code FROM \
                                temp.temp_Addresses \
                                  WHERE u_country_code = 'CA'  "
phone_numbers = connection.cursor().execute(phone_sql).fetchall()
for phone_number in phone_numbers:
    #print(phone_number[0])
    pattern = generate_pattern(str(phone_number[0]))
    #print(pattern)

    if pattern in pattern_counts:
        pattern_counts[pattern] += 1
    else:
        pattern_counts[pattern] = 1
data = {'Pattern': [], 'Count': []}

for pattern, count in pattern_counts.items():
    #formatted_pattern = re.sub(r'm+', 'm', pattern)  # Replace consecutive 'm's with a single 'm'
    data['Pattern'].append(pattern)
    data['Count'].append(count)

df = pd.DataFrame(data)
print(df)
df.to_csv('CA_TEMPaddress_postal_code.csv', index=False)

    Pattern    Count
0   ADA DAD  2982872
1    ADADAD    14702
2  ADA  DAD       14
3   ADAD AD        1
4  AD A DAD        1
5     ADADA        1


In [ ]:
!pip install phonenumbers

In [ ]:
!pip install --upgrade phonenumbers

In [ ]:
import phonenumbers
import pandas as pd
phone_sql = f"SELECT top 200000  u_postal_code FROM \
                                accounts.Addresses \
                                  WHERE dt_date_created > = DATEADD(YEAR,-2,GETDATE()) and u_country_code = 'CA' "
phone_numberss = connection.cursor().execute(phone_sql).fetchall()


invalid_count = 0
valid_count = 0
invalid_phones =[]
for phone_number in phone_numberss:
    z = phonenumbers.parse(phone_number[0], 'CA')
    #print(z)
    if phonenumbers.is_valid_number(z): 
        valid_count = valid_count+1
    else: 
        c = phonenumbers.parse(phone_number[0], 'US')
        if phonenumbers.is_valid_number(c): 
           valid_count = valid_count+1
        else:
           invalid_count = invalid_count+1
           invalid_phones.append(phone_number[0])
        
print(invalid_phones) 
#xyz = ['a','b', 'c','a']
df = pd.DataFrame(invalid_phones, columns=['invalid_phone'])
print(df)
df.to_csv('inavlidphn.csv', index=False)

# Perform a group-by operation and count occurrences
grouped_df = df.groupby('invalid_phone').size().reset_index(name='Count')

# Sort the DataFrame by the count column in descending order
sorted_df = grouped_df.sort_values(by='Count', ascending=False)

# Print the sorted DataFrame
print(sorted_df)
sorted_df.to_csv('invalid_phone_frequency.csv', index=False)


In [ ]:
print(df)


In [ ]:
print(invalid_count)

In [ ]:
for phone_number in invalid_phones:
    z = phonenumbers.parse(phone_number, 'CA')
    print("Phone CA:",phone_number, z)
    if phonenumbers.is_valid_number(z): 
        valid_count = valid_count+1
    else: 
        print("phone US: ",phone_number, c)
        c = phonenumbers.parse(phone_number, 'US')
        if phonenumbers.is_valid_number(c): 
           valid_count = valid_count+1
        else:
           invalid_count = invalid_count+1
           invalid_phones.append(phone_number[0])